<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/13-RAG/OllmaaOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu biraz hile, amacımız ollamaayı indirip termianlde çalıştırmak. Bunun için bir terminal  arayüzü edineceğiz. Bunu T4 GPU ile yap öneri.

Aşağıdaki koddan terminal çalıştır ve soınrasında biraz bekle az delaylı geliyor.

Terminalde ollama sistemini çalıştır

curl https://ollama.ai/install.sh | sh

Alttakiniki yazınca bir süre sonra duracak
ollama serve &

Durunca ir modeli indir ve kullanmaya başla

ollama run llama3.2:1b

sana send message diyecek yaz.


Altta bunu streoidli versiyonu var

xtermde
curl https://ollama.ai/install.sh | sh
ollama serve & ollama pull mistral
patlatarak direkt tekte indir kur aç mümkün

terminal dışında kullanmak için termianlden aşağı scroll at



In [ ]:
#Install and run xterm
!pip install colab-xterm
%load_ext colabxterm
%xterm